In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statistics
import csv
import glob


import os
import shutil
import pylab
from scipy.optimize import curve_fit
from scipy.special import erf
import ipywidgets as widgets
from sklearn.metrics import r2_score


path = '/Users/gomez/chem/EPFL/Fel experiments/R2lox/Alvra Beamtime/txtAlvra/FeMnR2lox_final/timing/'
path2 = '/Users/gomez/chem/EPFL/Fel experiments/R2lox/Alvra Beamtime/txtAlvra/FeMnR2lox_final/timing/'

os.chdir(path)
%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from scipy.optimize import curve_fit
from scipy.special import erf
from sklearn.metrics import r2_score


def irf_convolved_decay(t, t0, tau, sigma):
    term1 = np.exp(-(t - t0 - sigma ** 2 / (2 * tau)) / tau)
    term2 = erf((t - t0 - sigma ** 2 / tau) / (np.sqrt(2) * sigma))
    return 0.5 * term1 * (1 + term2)


def evaluate_fit(y_true, y_fit):
    residuals = y_true - y_fit
    sse = np.sum(residuals ** 2)
    rmse = np.sqrt(np.mean(residuals ** 2))
    r2 = r2_score(y_true, y_fit)
    return sse, rmse, r2, residuals


def plot_fit(t, y, y_fit, components, text_info, residuals, title="Fluorescence Fit"):
    plt.figure(figsize=(8, 5))
    plt.plot(t, y, 'o', label='Data', color='blue')
    plt.plot(t, y_fit, label='Total Fit', color='red')
    for comp, label, color in components:
        plt.plot(t, comp, '--', label=label, color=color)

    plt.xlabel('Time (fs)')
    plt.ylabel('Fluorescence Signal')
    plt.title(title)
    plt.legend()

    # Annotation
    box1, box2 = text_info
    props1 = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    props2 = dict(boxstyle='square', facecolor='white', alpha=0.2)

    plt.text(0.5, 0.85, box1, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=props1)
    plt.text(0.7, 0.05, box2, transform=plt.gca().transAxes, fontsize=9, verticalalignment='bottom', bbox=props2)

    plt.tight_layout()
    plt.show()

    # Residuals plot
    plt.figure()
    plt.plot(t, residuals, 'o', color='blue')
    plt.axhline(0, linestyle='--', color='gray')
    plt.xlabel('Time (fs)')
    plt.ylabel('Residuals')
    plt.title('Residuals')
    plt.tight_layout()
    plt.show()


def save_results_to_csv(data_dict, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Parameter", "Value"])
        for key, value in data_dict.items():
            writer.writerow([key, value])


def fit_2decay(t, y, p0, bounds, output_csv):
    def model(t, tau1, tau2, t0, A1, A2, sigma):
        return (A1 * irf_convolved_decay(t, t0, tau1, sigma) +
                A2 * irf_convolved_decay(t, t0, tau2, sigma))

    popt, pcov = curve_fit(model, t, y, p0=p0, bounds=bounds, maxfev=40000)
    y_fit = model(t, *popt)
    perr = np.sqrt(np.diag(pcov))
    tau1, tau2, t0, A1, A2, sigma = popt
    tau1_err, tau2_err, t0_err, A1_err, A2_err, sigma_err = perr

    fwhm = sigma * 2.3548
    sse, rmse, r2, residuals = evaluate_fit(y, y_fit)

    c1 = A1 * irf_convolved_decay(t, t0, tau1, sigma)
    c2 = A2 * irf_convolved_decay(t, t0, tau2, sigma)

    box1 = (f"t0: {t0:.2f} fs\n"
            f"Tau1: {tau1:.1f} fs\nTau2: {tau2 / 1000:.1f} ps\nFWHM: {fwhm:.2f} fs")
    box2 = (f"R²: {r2:.4f}\nSSE: {sse:.4f}\nRMSE: {rmse:.4f}")

    results = {
        'Tau1': f"{tau1:.4f} ± {tau1_err:.1f} fs",
        'Tau2': f"{tau2 / 1000:.4f} ± {tau2_err / 1000:.1f} ps",
        't0': f"{t0:.4f} ± {t0_err:.1f} fs",
        'A1': f"{A1:.4f}",
        'A2': f"{A2:.4f}",
        'Sigma': f"{sigma:.4f} ± {sigma_err:.2f} fs",
        'FWHM': f"{fwhm:.2f} fs",
        'R²': f"{r2:.4f}",
        'SSE': f"{sse:.4f}",
        'RMSE': f"{rmse:.4f}"
    }

    plot_fit(t, y, y_fit,
             components=[(c1, "Component 1", 'green'), (c2, "Component 2", 'orange')],
             text_info=(box1, box2), residuals=residuals)

    save_results_to_csv(results, output_csv)


def fit_3decay(t, y, p0, bounds, output_csv, fixed_tau3=None):
    def model(t, tau1, tau2, *params):
        if fixed_tau3 is not None:
            t0, A1, A2, A3, sigma = params
            tau3 = fixed_tau3
        else:
            tau3, t0, A1, A2, A3, sigma = params

        return (A1 * irf_convolved_decay(t, t0, tau1, sigma) +
                A2 * irf_convolved_decay(t, t0, tau2, sigma) +
                A3 * irf_convolved_decay(t, t0, tau3, sigma))

    popt, pcov = curve_fit(model, t, y, p0=p0, bounds=bounds, maxfev=40000)
    y_fit = model(t, *popt)
    perr = np.sqrt(np.diag(pcov))

    if fixed_tau3 is not None:
        tau1, tau2, t0, A1, A2, A3, sigma = popt
        tau3 = fixed_tau3
        tau1_err, tau2_err, t0_err, A1_err, A2_err, A3_err, sigma_err = perr
    else:
        tau1, tau2, tau3, t0, A1, A2, A3, sigma = popt
        tau1_err, tau2_err, tau3_err, t0_err, A1_err, A2_err, A3_err, sigma_err = perr

    fwhm = sigma * 2.3548
    sse, rmse, r2, residuals = evaluate_fit(y, y_fit)

    c1 = A1 * irf_convolved_decay(t, t0, tau1, sigma)
    c2 = A2 * irf_convolved_decay(t, t0, tau2, sigma)
    c3 = A3 * irf_convolved_decay(t, t0, tau3, sigma)

    box1 = (f"t0: {t0:.2f} fs\n"
            f"Tau1: {tau1:.1f} fs\nTau2: {tau2 / 1000:.1f} ps\n"
            f"Tau3: {tau3 / 1000:.1f} ps\nFWHM: {fwhm:.2f} fs")
    box2 = (f"R²: {r2:.4f}\nSSE: {sse:.4f}\nRMSE: {rmse:.4f}")

    results = {
        'Tau1': f"{tau1:.4f} ± {tau1_err:.1f} fs",
        'Tau2': f"{tau2 / 1000:.4f} ± {tau2_err / 1000:.1f} ps",
        'Tau3': f"{tau3 / 1000:.4f}" + ("" if fixed_tau3 else f" ± {tau3_err / 1000:.1f} ps"),
        't0': f"{t0:.4f} ± {t0_err:.1f} fs",
        'A1': f"{A1:.4f}",
        'A2': f"{A2:.4f}",
        'A3': f"{A3:.4f}",
        'Sigma': f"{sigma:.4f} ± {sigma_err:.2f} fs",
        'FWHM': f"{fwhm:.2f} fs",
        'R²': f"{r2:.4f}",
        'SSE': f"{sse:.4f}",
        'RMSE': f"{rmse:.4f}"
    }

    plot_fit(t, y, y_fit,
             components=[(c1, "Component 1", 'green'),
                         (c2, "Component 2", 'orange'),
                         (c3, "Component 3", 'purple')],
             text_info=(box1, box2), residuals=residuals)

    save_results_to_csv(results, output_csv)
    return popt, y_fit

